In [ ]:
import os
import json
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import optuna
from catboost import CatBoostRegressor, Pool

TRAIN_CSV = "train.csv"
OUT_DIR = "catboost_kfold_optuna"
N_SPLITS = 5
N_TRIALS = 30
SEED = 42

df = pd.read_csv(TRAIN_CSV)
target_col = "target"
X = df.drop(columns=[target_col])
y = df[target_col]
cat_cols = [col for col in X.columns if X[col].dtype == "object" or pd.api.types.is_categorical_dtype(X[col])]

base_params = {
    "loss_function": "RMSE",
    "task_type": "CPU",
    "max_depth": 6,
    "eval_metric": "RMSE",
    "learning_rate": 0.1,
    "grow_policy": "SymmetricTree",
    "l2_leaf_reg": 5,
    "random_strength": 1,
    "random_state": SEED,
    "verbose": False,
    "iterations": 10000
}

os.makedirs(OUT_DIR, exist_ok=True)
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
fold_results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y), start=1):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    def objective(trial):
        params = base_params.copy()
        params["max_depth"] = trial.suggest_int("max_depth", 4, 10)
        params["learning_rate"] = trial.suggest_float("learning_rate", 1e-3, 0.3, log=True)
        params["l2_leaf_reg"] = trial.suggest_int("l2_leaf_reg", 1, 20)
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0.0, 1.0)
        params["random_strength"] = trial.suggest_float("random_strength", 0.0, 20.0)
        params["subsample"] = trial.suggest_float("subsample", 0.5, 1.0)
        params["min_data_in_leaf"] = trial.suggest_int("min_data_in_leaf", 1, 100)
        params["border_count"] = trial.suggest_int("border_count", 32, 255)
        model = CatBoostRegressor(**params)
        train_pool = Pool(X_train, y_train, cat_features=cat_cols)
        val_pool = Pool(X_val, y_val, cat_features=cat_cols)
        model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=100, use_best_model=True, verbose=False)
        preds = model.predict(X_val)
        rmse = mean_squared_error(y_val, preds, squared=False)
        return rmse

    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=SEED))
    study.optimize(objective, n_trials=N_TRIALS)
    best_params = study.best_params
    final_params = base_params.copy()
    final_params.update(best_params)
    final_model = CatBoostRegressor(**final_params)
    train_pool_full = Pool(X_train, y_train, cat_features=cat_cols)
    val_pool_full = Pool(X_val, y_val, cat_features=cat_cols)
    final_model.fit(train_pool_full, eval_set=val_pool_full, early_stopping_rounds=100, use_best_model=True, verbose=False)
    val_preds = final_model.predict(X_val)
    val_rmse = mean_squared_error(y_val, val_preds, squared=False)

results_df = pd.DataFrame(fold_results)
results_df.to_csv(os.path.join(OUT_DIR, "cv_fold_results.csv"), index=False)
avg_rmse = results_df["rmse"].mean()


In [ ]:
import os
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import optuna
import lightgbm as lgb

TRAIN_CSV = "train.csv"
OUT_DIR = "lgb_kfold_optuna"
N_SPLITS = 5
N_TRIALS = 40
SEED = 42

df = pd.read_csv(TRAIN_CSV)
target_col = "target"
X = df.drop(columns=[target_col])
y = df[target_col]
cat_cols = [c for c in X.columns if X[c].dtype == "object" or pd.api.types.is_categorical_dtype(X[c])]
cat_feat_indices = [X.columns.get_loc(c) for c in cat_cols]

base_params = {
    "objective": "mse",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "random_state": SEED,
    "lambda_l2": 5.0,
    "learning_rate": 0.1,
    "num_leaves": 32,
    "extra_trees": True,
    "metric": "mse",
}

os.makedirs(OUT_DIR, exist_ok=True)
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
fold_results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y), start=1):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    def objective(trial):
        params = base_params.copy()
        params["num_leaves"] = trial.suggest_int("num_leaves", 8, 512)
        params["lambda_l1"] = trial.suggest_float("lambda_l1", 0.0, 10.0)
        params["lambda_l2"] = trial.suggest_float("lambda_l2", 0.0, 20.0)
        params["min_data_in_leaf"] = trial.suggest_int("min_data_in_leaf", 1, 500)
        params["feature_fraction"] = trial.suggest_float("feature_fraction", 0.4, 1.0)
        params["bagging_fraction"] = trial.suggest_float("bagging_fraction", 0.4, 1.0)
        params["bagging_freq"] = trial.suggest_int("bagging_freq", 0, 10)
        params["extra_trees"] = trial.suggest_categorical("extra_trees", [True, False])
        use_dart = trial.suggest_categorical("use_dart", [False, True])
        params["boosting_type"] = "dart" if use_dart else "gbdt"
        params["reg_sqrt"] = trial.suggest_categorical("reg_sqrt", [False, True])
        params["min_split_gain"] = trial.suggest_float("min_split_gain", 0.0, 5.0)
        params["learning_rate"] = trial.suggest_float("learning_rate", 1e-3, 0.3, log=True)
        model = lgb.LGBMRegressor(**params, n_estimators=10000)
        fit_params = {"eval_set": [(X_val, y_val)], "early_stopping_rounds": 100, "verbose": False}
        if len(cat_cols) > 0:
            fit_params["categorical_feature"] = cat_cols
        model.fit(X_train, y_train, **fit_params)
        preds = model.predict(X_val, num_iteration=model.best_iteration_)
        rmse = mean_squared_error(y_val, preds, squared=False)
        return rmse

    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=SEED))
    study.optimize(objective, n_trials=N_TRIALS)
    best_params = study.best_params
    final_params = base_params.copy()
    final_params.update(best_params)
    final_model = lgb.LGBMRegressor(**final_params, n_estimators=10000)
    fit_params = {"eval_set": [(X_val, y_val)], "early_stopping_rounds": 100, "verbose": False}
    if len(cat_cols) > 0:
        fit_params["categorical_feature"] = cat_cols
    final_model.fit(X_train, y_train, **fit_params)
    val_preds = final_model.predict(X_val, num_iteration=final_model.best_iteration_)
    val_rmse = mean_squared_error(y_val, val_preds, squared=False)

results_df = pd.DataFrame(fold_results)
results_df.to_csv(os.path.join(OUT_DIR, "lgb_cv_fold_results.csv"), index=False)
avg_rmse = results_df["rmse"].mean()


AutoGluon

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor

TRAIN_CSV = "train.csv"
TEST_CSV = "test.csv"
TIME_LIMIT = 600
PRED_OUTPUT = "test_predictions_autogluon.csv"

df = pd.read_csv(TRAIN_CSV)
target_col = "target"

X = df.drop(columns=[target_col])
y = df[target_col]
df[target_col] = y

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df[target_col], random_state=42)

predictor = TabularPredictor(label=target_col, problem_type="classification").fit(
    train_data=train_df,
    tuning_data=val_df,
    time_limit=TIME_LIMIT,
    presets="medium_quality_faster_train"
)

eval_results = predictor.evaluate(val_df)

test_df = pd.read_csv(TEST_CSV)
preds = predictor.predict(test_df)
probs = predictor.predict_proba(test_df)
out = test_df.copy()
out["prediction"] = preds
if isinstance(probs, pd.DataFrame):
    for col in probs.columns:
        out[f"prob_{col}"] = probs[col].values
else:
    out["probability"] = probs
out.to_csv(PRED_OUTPUT, index=False)


In [ ]:
# с указанием типов данных
from autogluon.tabular import TabularDataset, TabularPredictor
df = pd.read_csv("train.csv")
feature_types = {'user_id':'category', 'age':'int', 'comment':'text'}
predictor = TabularPredictor(label='target').fit(df, feature_types=feature_types)

3) Быстрые параметры fit() — что важно менять

time_limit — общий таймаут (секунды) на весь fit(); удобно для итераций/прототипов.

presets — наборы конфигураций (например, 'medium_quality_faster_train', 'best_quality', 'high_quality_fast_inference', extreme и т.д.). Presets задают: какие модели, bagging, stack levels, HPO defaults и пр. Начни с пресета для быстрой разработки, затем переключись на best_quality для финального прогона. 


hyperparameters — словарь, где ключи = модели (GBM, CAT, XGBoost, RF, NN_TORCH и т.д.), значения — либо список конфигураций, либо dict гиперпараметров (включая поисковые пространства через autogluon.core объекты). Пример см. ниже. 


hyperparameter_tune_kwargs — включить HPO (число trial'ов, стратегия поиска/шедьюлер). Пример: {'searcher':'random','scheduler':'local','num_trials':50} (точные ключи зависят от версии AutoGluon; searcher часто обязателен). 


ag_args_fit — внутренняя настройка обучения конкретной модели (num_cpus, num_gpus, max_rows, max_features и т.п.). Полезно для контроля ресурсов.

Масштабирование: если у тебя много строк/фич — контролируй max_rows/max_features и используешь num_cpus/num_gpus.

Когда ручной контроль нужен: при специфичных фичах/лейблах/пропусках — лучше подготовить фичи заранее и передать feature_types, либо отключать встроенные трансформации для некоторых моделей через hyperparameters/ag_args_fit.

In [ ]:
hyperparameters = {
    'GBM': {'num_boost_round': 1000, 'learning_rate': 0.03},
    'NN_TORCH': {'num_epochs': 10},
    'CAT': {'iterations': 1000},
    'KNN': {}, 'RF': {}
}
hpo = {'searcher':'random', 'scheduler':'local', 'num_trials': 20}
predictor = TabularPredictor(label='target').fit(
    train_data=train_df,
    time_limit=3600,
    presets='best_quality',
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hpo,
    ag_args_fit={'num_cpus':8, 'num_gpus':1}
)


num_bag_folds — число фолдов для bagging; по умолчанию >0 для многих пресетов. Увеличение повышает стабильность/качество, но пропорционально увеличивает время обучения и inference complexity.

num_bag_sets — сколько раз повторить bagging (repeats).

Если тебе важна inference-скорость, после тренировки можно вызвать predictor.collapse_ensemble() / refit_full() (или удалить модели, оставить самый сильный) — для резкого уменьшения времени inference с небольшим падением качества.

AutoGluon поддерживает sample_weight: можно передать имя столбца с весами при создании TabularPredictor (sample_weight='w') или в fit/evaluate. Также есть удобные значения 'auto_weight' и 'balance_weight'. Это полезно при сильном дисбалансе.

eval_metric можно задать при создании TabularPredictor(label=..., eval_metric='f1'). Подбирай метрику, релевантную задаче (AUC/F1/accuracy/mae и т.д.).

AutoGluon строит внутренние валидации/фолды; если хочешь control — передавай tuning_data (валидационная выборка) или указывай num_bag_folds/num_stack_levels.